In [5]:
!cd .. && make dataset && cd notebooks

>>> Downloading and saving data files...
Data files already downloaded.
>>> OK.



In [6]:
from codecarbon import OfflineEmissionsTracker

tracker = OfflineEmissionsTracker(save_to_file=False, country_iso_code="FRA")
tracker.start()

[codecarbon INFO @ 19:39:37] offline tracker init
[codecarbon INFO @ 19:39:37] [setup] RAM Tracking...
[codecarbon INFO @ 19:39:37] [setup] GPU Tracking...
[codecarbon INFO @ 19:39:37] No GPU found.
[codecarbon INFO @ 19:39:37] [setup] CPU Tracking...
[codecarbon ERROR @ 19:39:37] Unable to read Intel RAPL files for CPU power, we will use a constant for your CPU power. Please view https://github.com/mlco2/codecarbon/issues/244 for workarounds : [Errno 13] Permission denied: '/sys/class/powercap/intel-rapl/intel-rapl:1/energy_uj'
[codecarbon ERROR @ 19:39:37] Unable to read Intel RAPL files for CPU power, we will use a constant for your CPU power. Please view https://github.com/mlco2/codecarbon/issues/244 for workarounds : [Errno 13] Permission denied: '/sys/class/powercap/intel-rapl/intel-rapl:0/energy_uj'
[codecarbon INFO @ 19:39:37] Tracking Intel CPU via RAPL interface
[codecarbon ERROR @ 19:39:38] Unable to read Intel RAPL files for CPU power, we will use a constant for your CPU po

In [7]:
import json
import warnings
from pathlib import Path

import modin.pandas as pd
import pandas
import random
import plotly.io as pio
from pandas_profiling import ProfileReport
from tqdm import tqdm

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


pio.renderers.default = "notebook"


pd.options.plotting.backend = "plotly"


DATA_PATH = Path("../data")
FRAMES_JSON_PATH = Path(DATA_PATH, "raw/frames.json")

assets = {
    "intents": [
        {"name": "book"},
    ],
    "entities": [
        {"name": "or_city"},
        {"name": "dst_city"},
        {"name": "str_date"},
        {"name": "end_date"},
        {"name": "budget"},
    ],
    "examples": [],
}

In [8]:
%%time

raw_data = pd.read_json(FRAMES_JSON_PATH)

for turn in tqdm(raw_data["turns"]):
    for frame in turn:
        if frame["author"] == "wizard":
            continue

        entities = []

        for act in frame["labels"]["acts_without_refs"]:
            for arg in act["args"]:
                if arg["key"] == "intent" and arg["val"] == "book":
                    is_book = True

                if (
                    arg["key"] in [e["name"] for e in assets["entities"]]
                    and arg["val"] is not None
                    and frame["text"].find(arg["val"]) != -1
                ):
                    entity = {
                        "entityName": arg["key"],
                        "offset": frame["text"].index(arg["val"]),
                        "length": len(arg["val"]),
                    }
                    entities.append(entity)

        if len(entities) > 0:
            assets["examples"].append(
                {
                    "text": frame["text"],
                    "language": "en-us",
                    "intent": "book",
                    "entities": entities,
                    "dataset": "Train" if random.random() < 0.75 else "Test",
                }
            )

  0%|          | 0/1369 [00:00<?, ?it/s][codecarbon INFO @ 19:39:51] Energy consumed for RAM : 0.000336 kWh. RAM Power : 5.758007526397705 W
[codecarbon INFO @ 19:39:51] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 19:39:51] 0.000336 kWh of electricity used since the begining.
[codecarbon INFO @ 19:39:53] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.758007526397705 W
[codecarbon INFO @ 19:39:53] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 19:39:53] 0.000024 kWh of electricity used since the begining.
100%|██████████| 1369/1369 [00:09<00:00, 144.05it/s]

CPU times: user 10.4 s, sys: 868 ms, total: 11.3 s
Wall time: 18.6 s


In [9]:
body = {
    "api-version": "2021-11-01-preview",
    "metadata": {
        "name": "EmailProject",
        "description": "A test application",
        "type": "Conversation",
        "multilingual": True,
        "language": "en-us",
        "settings": {},
    },
    "assets": assets,
}


In [10]:
body

{'api-version': '2021-11-01-preview',
 'metadata': {'name': 'EmailProject',
  'description': 'A test application',
  'type': 'Conversation',
  'multilingual': True,
  'language': 'en-us',
  'settings': {}},
 'assets': {'intents': [{'name': 'book'}],
  'entities': [{'name': 'or_city'},
   {'name': 'dst_city'},
   {'name': 'str_date'},
   {'name': 'end_date'},
   {'name': 'budget'}],
  'examples': [{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
    'language': 'en-us',
    'intent': 'book',
    'entities': [{'entityName': 'dst_city', 'offset': 27, 'length': 8},
     {'entityName': 'or_city', 'offset': 41, 'length': 7},
     {'entityName': 'str_date', 'offset': 52, 'length': 25},
     {'entityName': 'budget', 'offset': 117, 'length': 4}],
    'dataset': 'Train'},
   {'text': 'Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 adults. For this trip, my budget would be 1900.',
    'l

In [11]:
tracker.stop()

[codecarbon INFO @ 19:39:57] Energy consumed for RAM : 0.000031 kWh. RAM Power : 5.758007526397705 W
[codecarbon INFO @ 19:39:57] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 19:39:58] 0.000031 kWh of electricity used since the begining.


1.684355111513264e-06

[codecarbon INFO @ 19:40:06] Energy consumed for RAM : 0.000360 kWh. RAM Power : 5.758007526397705 W
[codecarbon INFO @ 19:40:06] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 19:40:06] 0.000360 kWh of electricity used since the begining.
[codecarbon INFO @ 19:40:21] Energy consumed for RAM : 0.000384 kWh. RAM Power : 5.758007526397705 W
[codecarbon INFO @ 19:40:21] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 19:40:21] 0.000384 kWh of electricity used since the begining.
[codecarbon INFO @ 19:40:36] Energy consumed for RAM : 0.000408 kWh. RAM Power : 5.758007526397705 W
[codecarbon INFO @ 19:40:36] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 0.0 W
[codecarbon INFO @ 19:40:36] 0.000408 kWh of electricity used since the begining.
[codecarbon INFO @ 19:40:51] Energy consumed for RAM : 0.000432 kWh. RAM Power : 5.758007526397705 W
[codecarbon INFO @ 19:40:51] Energy consumed for all CPUs :